In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# модели
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.svm import SVC, OneClassSVM, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.cluster import KMeans

# метрики
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

import warnings
import math
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
from statistics import mean, median
from time import sleep
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm
import numpy as np
import scipy.stats as stats
from scipy.stats import gmean
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import emoji
import torch
from slovnet.model.emb import NavecEmbedding
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/konstantin_sukhin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/konstantin_sukhin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/konstantin_sukhin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def preprocess_text(text):
    #text = emoji.get_emoji_regexp().sub("", text) # удаление эмодзи
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\d+', '', text) # удаление цифр
    text = re.sub(r'\s+', ' ', text).strip() # удаление лишних пробелов
    tokens = word_tokenize(text) # Токенизация
    tokens = [token.lower() for token in tokens] # Приведение к нижнему регистру
    lemmatizer = WordNetLemmatizer() # Лемматизация
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    table = str.maketrans('', '', string.punctuation + '’') # Очистка от знаков пунктуации и других символов
    tokens = [token.translate(table) for token in tokens]
    tokens = [token for token in tokens if len(token) > 0] # Удаление пустых токенов
    return tokens

In [58]:
df = pd.read_excel('detection_df.xlsx')
df['target'] = df['value'].replace({'Fake':1,"True":0})
clear = df.copy()
clear['review'] = clear['review'].apply(preprocess_text)
clear = clear.drop(columns=['Unnamed: 0','headphones_id','sorted','value','var_other_review_len','std_similarity_of_client_reviews'])

# Изучение особенностей клиентов с поддельными отзывами.

In [59]:
comp = df.groupby(['headphones_id'])['target','rating'].agg(['count','sum'])
comp['share'] = comp['target']['sum']/comp['target']['count']
comp['avg'] = comp['rating']['sum']/comp['rating']['count']
comp

<ipython-input-59-621e31c298b6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  comp = df.groupby(['headphones_id'])['target','rating'].agg(['count','sum'])


target      rating           share       avg
               count  sum  count   sum                    
headphones_id                                             
8601593           23   16     23   115  0.695652  5.000000
9943236           29    8     29    77  0.275862  2.655172
22003393         206  113    206   902  0.548544  4.378641
75212617          17    7     17    67  0.411765  3.941176
75640333          63   47     63   284  0.746032  4.507937
86283646          61   35     61   279  0.573770  4.573770
91198168         327  237    327  1486  0.724771  4.544343
93199503         153  109    153   712  0.712418  4.653595
100339707         74   59     74   354  0.797297  4.783784
100752412         59   30     59   273  0.508475  4.627119
100817270        334  215    334  1512  0.643713  4.526946
103487056         38   11     38   136  0.289474  3.578947
109266137         96   54     96   449  0.562500  4.677083
110259188         60   29     60   265  0.483333  4.416667
113148910        259  178    259  1216  0.687259  4.694981
119906733        151   90    151   698  0.596026  4.622517
125406299         74   47     74   341  0.635135  4.608108
125436671         53   26     53   243  0.490566  4.584906
138216345         63   48     63   293  0.761905  4.650794
139140790        469  345    469  2179  0.735608  4.646055
139823386        494  341    494  2275  0.690283  4.605263
143874380        166  126    166   757  0.759036  4.560241
146523913         43   19     43   175  0.441860  4.069767
148056502         83   56     83   371  0.674699  4.469880
150048947        355  185    355  1535  0.521127  4.323944
150136196        130   83    130   601  0.638462  4.623077
150606036        256  173    256  1270  0.675781  4.960938
151563780         80   49     80   390  0.612500  4.875000
151973186         94   58     94   420  0.617021  4.468085
154532809        137  101    137   646  0.737226  4.715328

In [60]:
comp['share'].min(),comp['share'].max()

(0.27586206896551724, 0.7972972972972973)

In [61]:
clear['target'].mean()

0.6510006746120981

In [62]:
fake_df = df[df['target']==1]
true_df = df[df['target']==0]

In [63]:
from statsmodels.stats.weightstats import ztest 

In [71]:
table_1 = pd.DataFrame(columns=['Признаки','Среднее значение (спамеры)','Среднее значение (подлинные)','p-value'])
count = 0
for i in ['months_on_site','reviews_number','main_review_len','rating',
         'minutes_between_reviews','date_std','share_of_negative_ratings','similarity_of_client_reviews','similarity_of_main_reviews',
         'median_length_of_other_reviews','median_number_of_photos',
         'var_other_rate','share_similarity_of_products']:
    
    table_1.loc[count] = [i,round(fake_df[i].mean(),2),round(true_df[i].mean(),2),round(ztest(fake_df[i],true_df[i])[1],2)]
    count +=1

In [72]:
table_1

,Признаки,Среднее значение (спамеры),Среднее значение (подлинные),p-value
0,months_on_site,18.03,18.61,0.32
1,reviews_number,15.01,9.59,0.00
2,main_review_len,115.19,101.83,0.00
3,rating,4.95,3.86,0.00
4,minutes_between_reviews,25957.76,51531.67,0.00
5,date_std,43648.54,48328.89,0.04
6,share_of_negative_ratings,0.06,0.20,0.00
7,similarity_of_client_reviews,0.21,0.19,0.01
8,similarity_of_main_reviews,0.43,0.33,0.00
9,median_length_of_other_reviews,72.36,70.47,0.28


In [73]:
table_1['Признаки'] = ['Количество месяцев на сайте у клиента',
                       'Количество всего отзывов у клиента',
                       'Длина отзыва',
                       'Рейтинг отзыва',
                       'Медианное количество минут между всеми отзывами клиента',
                       'Стандартное отклонение минут между отзывами клиента ',
                       'Доля отзывов с негативным рейтингом (1-3)',
                       'Уровень схожести отзывов клиента между собой ',
                       'Похожесть отзыва клиента, на другие отзывы на этот товар ',
                       'Медианное значение длины отзывов у клиента ',
                       'Медианное значение количества фотографий у клиента ',
                       'Дисперсия рейтингов клиента ',
                       'Доля товаров, совпадающих у других клиентов, купивших этот же товар']

In [76]:
table_1

,Признаки,Среднее значение (спамеры),Среднее значение (подлинные),p-value
0,Количество месяцев на сайте у клиента,18.03,18.61,0.32
1,Количество всего отзывов у клиента,15.01,9.59,0.00
2,Длина отзыва,115.19,101.83,0.00
3,Рейтинг отзыва,4.95,3.86,0.00
4,Медианное количество минут между всеми отзывами клиента,25957.76,51531.67,0.00
5,Стандартное отклонение минут между отзывами клиента,43648.54,48328.89,0.04
6,Доля отзывов с негативным рейтингом (1-3),0.06,0.20,0.00
7,Уровень схожести отзывов клиента между собой,0.21,0.19,0.01
8,"Похожесть отзыва клиента, на другие отзывы на этот товар",0.43,0.33,0.00
9,Медианное значение длины отзывов у клиента,72.36,70.47,0.28


In [10]:
clear_desc = clear.describe()
for i in clear_desc.columns:
    clear_desc[i] = clear_desc[i].round(2)

In [11]:
clear_desc = clear_desc.T
clear_desc = clear_desc.drop(columns='count')
clear_desc

,mean,std,min,25%,50%,75%,max
months_on_site,18.23,18.56,1.0,8.00,12.00,24.00,168.00
reviews_number,13.12,20.92,1.0,3.00,6.00,14.00,311.00
main_review_len,110.53,104.49,4.0,42.00,83.00,144.00,999.00
rating,4.57,1.09,1.0,5.00,5.00,5.00,5.00
minutes_between_reviews,34883.04,104504.46,0.0,161.00,9808.00,25432.00,2141428.00
date_std,45281.98,73571.89,0.0,4817.00,20683.00,51119.50,858585.00
share_of_negative_ratings,0.11,0.22,0.0,0.00,0.00,0.12,1.00
similarity_of_main_reviews,0.39,0.27,0.0,0.12,0.42,0.63,0.94
similarity_of_client_reviews,0.20,0.24,0.0,0.00,0.13,0.30,1.00
std_similarity_of_client_reviews,0.12,0.08,0.0,0.07,0.14,0.18,0.45


In [75]:
#clear_desc.to_excel('tototo.xlsx')
pd.set_option('display.max_colwidth', None)
clear_desc_c = pd.read_excel('tototo.xlsx')
clear_desc_c.rename(columns={'Unnamed: 0':'Признаки'})

,Признаки,Среднее значение,Стандартное отклонение,Минимум,25% квантиль,50% квантиль,75% квантиль,Максимум
0,Количество месяцев на сайте у клиента,18.23,18.56,1,8.00,12.00,24.00,168.00
1,Количество всего отзывов у клиента,13.12,20.92,1,3.00,6.00,14.00,311.00
2,Длина отзыва,110.53,104.49,4,42.00,83.00,144.00,999.00
3,Рейтинг отзыва,4.57,1.09,1,5.00,5.00,5.00,5.00
4,Медианное количество минут между всеми отзывами клиента,34883.04,104504.46,0,161.00,9808.00,25432.00,2141428.00
5,Стандартное отклонение минут между отзывами клиента,45281.98,73571.89,0,4817.00,20683.00,51119.50,858585.00
6,Доля отзывов с негативным рейтингом (1-3) среди всех отзывов клиента,0.11,0.22,0,0.00,0.00,0.12,1.00
7,"Похожесть отзыва клиента, на другие отзывы на этот товар",0.39,0.27,0,0.12,0.42,0.63,0.94
8,Уровень схожести отзывов клиента между собой,0.20,0.24,0,0.00,0.13,0.30,1.00
9,Медианное значение длины отзывов у клиента,71.70,55.37,6,39.00,62.00,86.00,995.00


# Подготовка и обучение моделей

In [13]:
text_embd = {}
for i in tqdm(clear['main_id'].unique()):    
    for j in clear.loc[clear['main_id']==i].index:
        text_embd_list = []
        for k in clear.loc[(clear.index==j)]['review'][j]:
            try: 
                text_embd_list.append(navec[k])
            except KeyError:
                continue
        if len(text_embd_list) == 0:
            pass
        else:
            mean_arr = np.mean(np.array(text_embd_list), axis=0)    
    text_embd.update({i: mean_arr})

100%|██████████| 4447/4447 [00:03<00:00, 1368.55it/s]


In [14]:
arrays = pd.DataFrame(text_embd).T.reset_index()
arrays

,index,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,139140790_949,-0.008525,-0.298617,-0.163041,0.010562,0.025108,-0.078944,0.117707,0.083552,0.060948,...,-0.055099,-0.058178,-0.036105,0.034979,-0.009179,-0.251608,0.007796,-0.255358,-0.125636,0.144480
1,113148910_527,0.054108,-0.122081,0.212901,-0.059517,-0.029191,-0.224025,0.397066,-0.102836,0.074497,...,0.084085,-0.277620,-0.096388,0.032181,0.034327,-0.202961,-0.366355,-0.571341,0.001537,0.169616
2,100817270_3481,0.000693,-0.182865,-0.276867,0.146147,-0.013100,-0.135788,0.151648,0.139120,0.169354,...,0.119999,0.034584,0.019883,0.077687,-0.041116,-0.313853,0.181792,-0.243730,-0.116130,0.399321
3,100817270_3819,0.045901,-0.120565,0.065262,0.336097,-0.100133,-0.359495,0.323829,0.021193,0.081585,...,0.032177,-0.476555,-0.004042,0.350212,0.002482,-0.285199,0.129337,-0.451944,-0.008566,0.113222
4,143874380_2044,-0.031632,-0.319485,-0.011594,-0.083991,-0.178352,-0.203103,0.188321,0.250263,0.178247,...,0.083395,-0.096229,-0.144180,0.036940,0.036881,-0.187580,0.128051,-0.227499,-0.028417,0.223107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4442,109266137_3911,-0.091533,-0.154407,-0.295144,-0.021771,0.040267,-0.222243,0.096137,0.162183,-0.087281,...,0.095537,0.473995,-0.033041,0.289569,-0.327162,-0.148973,-0.065017,0.204347,-0.019414,-0.049868
4443,125406299_4076,-0.093011,-0.051215,-0.177089,0.052424,0.146642,0.072435,0.025110,0.215731,0.216424,...,-0.006469,-0.054636,0.041526,0.101446,0.005896,-0.068884,0.094600,-0.034316,0.149689,0.028599
4444,125406299_4088,-0.010136,-0.230925,-0.128986,-0.054179,0.078767,0.042010,0.092126,0.098790,0.202100,...,-0.047305,-0.048059,0.055103,-0.027171,-0.180254,-0.294454,0.008948,-0.090994,0.087710,0.033454
4445,148056502_4178,-0.087266,-0.053218,-0.230607,0.283439,0.089981,-0.055759,0.320386,-0.151021,0.177617,...,0.132253,-0.192571,-0.018133,0.076700,-0.150206,-0.205033,0.081192,-0.396292,-0.124573,0.532887


In [15]:
try_df = clear.merge(arrays,left_on='main_id',right_on='index',how='inner')
try_df = try_df.drop(columns=['main_id','index','review'])
X = try_df.drop(columns='target')
y = try_df['target']

In [17]:
scaler = preprocessing.StandardScaler()
X_sc = scaler.fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.25, random_state=72, stratify=y)

### С учителем

In [19]:
all_results = []
def score_model(model, param_grid, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    print("Best parameters: {}".format(grid_search.best_params_))
    
    y_pred = grid_search.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print("acc: {}".format(acc))
    print("f1: {}".format(f1))
    return [acc,f1]

#### CatBoost

In [21]:
all_results.append(['CatBoost'] + score_model(CatBoostClassifier(verbose=False),
             {'learning_rate': [0.1, 0.01, 0.001],
              'depth': [3, 5, 7],
              'iterations': [50,100,500]}))

Best parameters: {'depth': 5, 'iterations': 500, 'learning_rate': 0.1}
acc: 0.8129496402877698
f1: 0.8647594278283486


#### Лес

In [22]:
all_results.append(['RandomForest'] + score_model(RandomForestClassifier(verbose=False, random_state=72),
                                                 {'n_estimators': [50, 100, 300],
                                                  'max_depth': [3, 5, 7],
                                                  'min_samples_split': [2, 5, 10]}))

Best parameters: {'max_depth': 7, 'min_samples_split': 5, 'n_estimators': 50}
acc: 0.7823741007194245
f1: 0.8527980535279807


#### SVC

In [24]:
all_results.append(['SVM'] + score_model(SVC(probability=True, random_state=72),
                                            {'C': [0.1, 1, 10],
                                             'kernel': ['linear', 'rbf', 'poly']}))

Best parameters: {'C': 1, 'kernel': 'rbf'}
acc: 0.7598920863309353
f1: 0.837492391965916


#### logistic regression

In [25]:
all_results.append(['Logistic'] + score_model(LogisticRegression(random_state=72,max_iter=2000),{
                                                                    'penalty': ['l1', 'l2'],
                                                                    'C': [0.1, 1, 10],
                                                                    'solver': ['liblinear']}))

Best parameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
acc: 0.783273381294964
f1: 0.8487131198995606


In [26]:
pd.DataFrame(all_results, columns=['Метод', 'Accuracy', 'F1-Score'])\
    .sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

,Метод,Accuracy,F1-Score
0,CatBoost,0.812950,0.864759
1,Logistic,0.783273,0.848713
2,RandomForest,0.782374,0.852798
3,SVM,0.759892,0.837492


### Без учителя

In [27]:
def score_model_unsupervised(model, X=X, y=y):
    y = y * 2 - 1
    
    model.fit(X)
    y_pred = model.predict(X)
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X)[:, 1]
        print('ROC-AUC:', roc_auc_score(y, y_proba))
    print('F-SCORE:', f1_score(y, y_pred))
    print('ACCURACY:', accuracy_score(y, y_pred))
    
    return [accuracy_score(y, y_pred), f1_score(y, y_pred)]

#### OneClassSVM

In [28]:
all_results.append(['OSVM'] + score_model_unsupervised(OneClassSVM()))

F-SCORE: 0.6611957796014067
ACCURACY: 0.6100742073307848


#### Isolation Forest

In [29]:
all_results.append(['Isolation Forest'] + score_model_unsupervised(IsolationForest(random_state=999)))

F-SCORE: 0.7832699619771862
ACCURACY: 0.6539239937036204


### Результаты

In [30]:
pd.DataFrame(all_results, columns=['Метод', 'Accuracy', 'F1-Score'
                                  ])\
    .sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

,Метод,Accuracy,F1-Score
0,CatBoost,0.812950,0.864759
1,Logistic,0.783273,0.848713
2,RandomForest,0.782374,0.852798
3,SVM,0.759892,0.837492
4,Isolation Forest,0.653924,0.783270
5,OSVM,0.610074,0.661196
